In [2]:
import weaviate

# Tạo client kết nối tới Weaviate
client = weaviate.Client("http://127.0.0.1:80")

# Lấy schema hiện tại
schema = client.schema.get()

print("Current Schema:")
print(schema)


Current Schema:
{'classes': [{'class': 'Document', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'multiTenancyConfig': {'autoTenantActivation': False, 'autoTenantCreation': False, 'enabled': False}, 'properties': [{'dataType': ['text'], 'indexFilterable': True, 'indexRangeFilters': False, 'indexSearchable': True, 'name': 'content', 'tokenization': 'word'}], 'replicationConfig': {'asyncEnabled': False, 'factor': 1}, 'shardingConfig': {'actualCount': 1, 'actualVirtualCount': 128, 'desiredCount': 1, 'desiredVirtualCount': 128, 'function': 'murmur3', 'key': '_id', 'strategy': 'hash', 'virtualPerPhysical': 128}, 'vectorIndexConfig': {'bq': {'enabled': False}, 'cleanupIntervalSeconds': 300, 'distance': 'cosine', 'dynamicEfFactor': 8, 'dynamicEfMax': 500, 'dynamicEfMin': 100, 'ef': -1, 'efConstruction': 128, 'flatSearchCutoff': 40000, 'maxConnections': 32, 'pq': {'bitCompression': Fals

In [3]:
import weaviate

# Tạo client kết nối tới Weaviate
client = weaviate.Client("http://127.0.0.1:80")

# Lấy danh sách các đối tượng trong class 'AIO_Doc'
result = client.data_object.get(class_name='AIO_Doc')

print(f"Objects in class 'AIO_Doc':")
for obj in result['objects']:
    print(obj)


Objects in class 'AIO_Doc':


TypeError: 'NoneType' object is not subscriptable

In [5]:
import weaviate
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Kết nối đến Weaviate instance trên Kubernetes
client = weaviate.Client("http://127.0.0.1:80")

# Định nghĩa model và tokenizer từ Hugging Face
model_name = "BAAI/bge-small-en-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

index_name = "AIO_Doc"

# Hàm để tạo embedding từ văn bản
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

# Tạo VectorStoreIndex nếu chưa tồn tại
if not client.schema.exists(index_name):
    schema = {
        "classes": [
            {
                "class": index_name,
                "description": "A class representing a document with text data",
                "vectorIndexType": "hnsw",
                "vectorIndexConfig": {
                    "efConstruction": 128,
                    "M": 16
                },
                "properties": [
                    {
                        "name": "text",
                        "dataType": ["text"],
                        "description": "The content of the document"
                    },
                    {
                        "name": "vector",
                        "dataType": ["number[]"],
                        "description": "The embedding vector of the document"
                    },
                    {
                        "name": "metadata",
                        "dataType": ["string"],
                        "description": "Metadata of the document"
                    }
                ]
            }
        ]
    }
    client.schema.create(schema)

# Hàm để thực hiện truy vấn và trả về các đoạn văn tương đương
def retrieve_similar_docs(question, top_k=5):
    question_embedding = embed_text(question).tolist()
    
    near_vector = {
        "vector": question_embedding,
        "certainty": 0.7
    }

    result = client.query.get(index_name, ["text", "metadata"]) \
        .with_near_vector(near_vector) \
        .with_limit(top_k) \
        .do()
    
    return result['data']['Get'][index_name]

# Thực hiện truy vấn với câu hỏi đầu vào
question = "mamba là gì?"
results = retrieve_similar_docs(question)

# Hiển thị kết quả
for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(f"Text: {result['text']}")
    print(f"Metadata: {result['metadata']}")
    print("\n")

print("Retrieval completed successfully.")


/home/duong/anaconda3/envs/rag/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/duong/anaconda3/envs/rag/lib/python3.9/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.6. The latest version is 4.7.1.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(
/home/duong/anaconda3/envs/rag/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Retrieval completed successfully.


In [4]:
import weaviate
import json

client = weaviate.Client("http://127.0.0.1:80")
response = (
    client.query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_hybrid(
      query="food",  # Query string
      properties=["question", "answer"],  # Searched properties
      vector=None  # Manually provide a vector; if not, Weaviate will vectorize the query string
    )
    .with_additional(["score", "explainScore"])  # Include score & explainScore in the response
    .with_limit(3)
    .do()
  )


print(json.dumps(response, indent=2))

{
  "errors": [
    {
      "locations": [
        {
          "column": 6,
          "line": 1
        }
      ],
      "message": "Cannot query field \"JeopardyQuestion\" on type \"GetObjectsObj\".",
      "path": null
    }
  ]
}


In [ ]:
git config --global user.email "nthaiduong83@gmail.com"
git config --global user.name "ntduong"